# 🛒 Regras de Associação (Apriori) — Cross-sell de Produtos
**Pós-Graduação BI & Analytics · ML Aplicado a Decisões de Negócio**

---

### 📡 Sinal de Negócio
> A segmentação revelou que clientes do Cluster de Risco têm `num_produtos_distintos = 1.3` — compram quase só um produto.  
> O diretor comercial pergunta: **existem produtos que costumam ser comprados juntos?**  
> Se sim, podemos criar ofertas de cross-sell automáticas no momento do pedido.

### 🏗️ O que vamos construir
Regras de associação que identificam padrões de co-ocorrência entre produtos.  
A saída é uma tabela de recomendações **ranqueadas por lift**.

---
### Roteiro
1. Importar e explorar a base transacional
2. Pré-processamento: pivot para matriz binária (passo crítico)
3. Calcular Apriori (itens frequentes)
4. Gerar regras de associação
5. Filtrar e ranquear pelo lift
6. Visualizar as regras
7. Saída acionável: tabela de cross-sell

In [ ]:
# ── CÉLULA PRONTA: imports ─────────────────────────────────────────
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# instalar mlxtend se necessário
try:
    from mlxtend.frequent_patterns import apriori, association_rules
    from mlxtend.preprocessing import TransactionEncoder
    print('✅ mlxtend carregado')
except ImportError:
    print('Instalando mlxtend...')
    import subprocess
    subprocess.run(['pip', 'install', 'mlxtend', '-q'])
    from mlxtend.frequent_patterns import apriori, association_rules
    from mlxtend.preprocessing import TransactionEncoder
    print('✅ mlxtend instalado e carregado')

plt.rcParams.update({'figure.dpi': 120, 'axes.spines.top': False, 'axes.spines.right': False})
print('✅ Bibliotecas prontas')

In [ ]:
# ── CÉLULA PRONTA: carregar base transacional ──────────────────────
URL = 'https://raw.githubusercontent.com/SEU_USUARIO/ml-bi-analytics-aula/main/data/transacoes_itens.csv'
# df = pd.read_csv('transacoes_itens.csv')  # local
df = pd.read_csv(URL)
print(f'Shape: {df.shape}')
print(f'Transações únicas: {df.id_transacao.nunique():,}')
print(f'Clientes únicos: {df.id_cliente.nunique():,}')
print(f'Produtos únicos: {df.produto.nunique()}')
print(f'Período: {df.data_pedido.min()} → {df.data_pedido.max()}')
df.head(8)

In [ ]:
# ── CÉLULA PRONTA: EDA da base transacional ───────────────────────
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Frequência por produto
freq_prod = df.groupby('produto')['id_transacao'].nunique().sort_values(ascending=True)
axes[0].barh(freq_prod.index, freq_prod.values, color='#2f6ec8')
axes[0].set(title='Transações por Produto', xlabel='Nº de Transações')

# Itens por transação
itens_por_tx = df.groupby('id_transacao')['produto'].count()
axes[1].hist(itens_por_tx, bins=range(1, 8), color='#2a8c5a', edgecolor='white', align='left')
axes[1].set(title='Itens por Transação', xlabel='Nº de Produtos', ylabel='Frequência')

# Volume por categoria
vol_cat = df.groupby('categoria')['valor_item'].sum().sort_values()
axes[2].barh(vol_cat.index, vol_cat.values, color='#7c3dc8')
axes[2].set(title='Faturamento por Categoria', xlabel='R$')

plt.tight_layout()
plt.show()
print(f'\nMédia de itens por transação: {itens_por_tx.mean():.1f}')

## 2. Pré-Processamento: Pivot para Matriz Binária

> ⚠️ **Este é o passo mais importante do Apriori e o mais esquecido.**  
> A base está em formato *longo* (1 linha = 1 produto de 1 pedido).  
> O Apriori precisa de formato *largo*: 1 linha = 1 pedido, 1 coluna = 1 produto, valor = 0/1.

In [ ]:
# ╔═══════════════════════════════════════════════════════════╗
# ║  COMPLETE AQUI — Pivot para matriz binária transação×produto  ║
# ╚═══════════════════════════════════════════════════════════╝
# Dica: df.groupby(['id_transacao','produto'])['qtd'].sum().unstack(fill_value=0)
# Depois: converter para 0/1 com .applymap(lambda x: 1 if x > 0 else 0)
# (ou: basket > 0 para versão mais limpa)

basket = df.groupby([___, ___])['qtd'].sum().unstack(fill_value=___)
basket = basket.applymap(lambda x: ___ if x > 0 else ___).astype(bool)

print(f'Matriz gerada: {basket.shape}')
print(f'  {basket.shape[0]:,} transações × {basket.shape[1]} produtos')
print(f'  Densidade: {basket.values.mean():.1%}')
basket.head()

## 3. Calcular Itemsets Frequentes (Apriori)

In [ ]:
# ╔══════════════════════════════════════════════════════╗
# ║  COMPLETE AQUI — Rodar apriori()                   ║
# ╚══════════════════════════════════════════════════════╝
# Dica: apriori(basket, min_support=0.02, use_colnames=True)
# min_support: proporção de transações onde o itemset aparece
# Tente 0.01 se retornar poucos itemsets

frequent_items = apriori(basket, min_support=___, use_colnames=True)
frequent_items['n_items'] = frequent_items['itemsets'].apply(len)

print(f'Itemsets frequentes encontrados: {len(frequent_items):,}')
print('\nDistribuição por tamanho:')
print(frequent_items['n_items'].value_counts().sort_index().to_string())
print('\nTop 10 por suporte:')
frequent_items.sort_values('support', ascending=False).head(10)

## 4. Gerar Regras de Associação

In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  COMPLETE AQUI — Gerar regras e filtrar por lift > 1.2    ║
# ╚══════════════════════════════════════════════════════════════╝
# Dica: association_rules(frequent_items, metric='lift', min_threshold=1.2)
# Depois: filtrar confidence >= 0.30

rules = association_rules(frequent_items, metric=___, min_threshold=___)
rules = rules[rules['confidence'] >= ___].copy()

# Formatar para leitura
rules['antecedents_str'] = rules['antecedents'].apply(lambda x: ', '.join(sorted(x)))
rules['consequents_str'] = rules['consequents'].apply(lambda x: ', '.join(sorted(x)))

print(f'Regras geradas: {len(rules):,}')
print('\nTop 10 por lift:')
rules.sort_values('lift', ascending=False)[
    ['antecedents_str','consequents_str','support','confidence','lift']
].head(10)

## 5. Visualizar — Scatter Suporte × Confiança

In [ ]:
# ── CÉLULA PRONTA: scatter das regras ─────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# Scatter suporte × confiança, tamanho = lift
scatter = axes[0].scatter(
    rules['support'], rules['confidence'],
    s=rules['lift'] * 30, c=rules['lift'],
    cmap='RdYlGn', alpha=0.7, edgecolors='white', lw=0.5
)
plt.colorbar(scatter, ax=axes[0], label='Lift')
axes[0].set(title='Regras de Associação\n(tamanho = lift)',
            xlabel='Suporte', ylabel='Confiança')
axes[0].axhline(0.4, color='#888', lw=1, linestyle='--', label='conf=0.4')
axes[0].legend(fontsize=8)

# Top 15 regras por lift — gráfico de barras
top15 = rules.nlargest(15, 'lift')
labels = [f"{r['antecedents_str']} →\n{r['consequents_str']}"
          for _, r in top15.iterrows()]
cores  = plt.cm.RdYlGn(np.linspace(0.4, 0.9, 15))
axes[1].barh(range(len(top15)), top15['lift'].values, color=cores[::-1])
axes[1].set(yticks=range(len(top15)), yticklabels=labels,
            title='Top 15 Regras por Lift', xlabel='Lift')
axes[1].axvline(1, color='#888', lw=1, linestyle='--')
axes[1].tick_params(axis='y', labelsize=7)

plt.tight_layout()
plt.show()

print('\n💡 Lift > 1: os produtos ocorrem juntos mais do que o acaso esperaria.')
print('   Lift = 1: independentes. Lift < 1: substitutos.')

In [ ]:
# ── CÉLULA PRONTA: matriz de lift entre produtos ──────────────────
prods = sorted(df['produto'].unique())
matrix = pd.DataFrame(1.0, index=prods, columns=prods)

for _, row in rules.iterrows():
    if len(row['antecedents']) == 1 and len(row['consequents']) == 1:
        a = list(row['antecedents'])[0]
        c = list(row['consequents'])[0]
        if a in prods and c in prods:
            matrix.loc[a, c] = row['lift']

fig, ax = plt.subplots(figsize=(10, 7))
im = ax.imshow(matrix.values, cmap='RdYlGn', vmin=0.5, vmax=3.5)
ax.set(xticks=range(len(prods)), yticks=range(len(prods)),
       xticklabels=[p.replace('Produto ','') for p in prods],
       yticklabels=[p.replace('Produto ','') for p in prods])
plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

for i in range(len(prods)):
    for j in range(len(prods)):
        val = matrix.values[i, j]
        if val != 1.0:
            ax.text(j, i, f'{val:.1f}', ha='center', va='center', fontsize=8)

plt.colorbar(im, ax=ax, label='Lift', shrink=0.8)
ax.set_title('Matriz de Lift entre Produtos\n(verde = complementares, vermelho = substitutos)',
             fontweight='bold')
plt.tight_layout()
plt.show()

## 7. Saída Acionável — Tabela de Cross-sell

In [ ]:
# ── CÉLULA PRONTA: tabela de recomendações ────────────────────────
tabela_crosssell = rules[
    (rules['lift'] > 1.5) &
    (rules['confidence'] >= 0.35) &
    (rules['support'] >= 0.01)
].sort_values('lift', ascending=False)[[
    'antecedents_str', 'consequents_str',
    'support', 'confidence', 'lift'
]].copy()

tabela_crosssell.columns = [
    'Se comprou', 'Recomendar',
    'Suporte', 'Confiança', 'Lift'
]
tabela_crosssell = tabela_crosssell.reset_index(drop=True)
tabela_crosssell['Suporte']    = tabela_crosssell['Suporte'].map('{:.1%}'.format)
tabela_crosssell['Confiança']  = tabela_crosssell['Confiança'].map('{:.1%}'.format)
tabela_crosssell['Lift']       = tabela_crosssell['Lift'].round(2)

print(f'Regras de cross-sell qualificadas: {len(tabela_crosssell)}')
print('\nTabela final para o time comercial:')
print(tabela_crosssell.to_string(index=False))

tabela_crosssell.to_csv('regras_crosssell.csv', index=False)
print('\n✅ regras_crosssell.csv exportada')
print('\n💡 Integrar essa tabela no CRM:')
print('   Quando cliente adicionar produto X ao carrinho → sugerir Y automaticamente.')

In [ ]:
# ── CÉLULA PRONTA: conexão com sistemas de recomendação ───────────
print('='*60)
print('DA REGRA DE ASSOCIAÇÃO AO SISTEMA DE RECOMENDAÇÃO')
print('='*60)
print()
print('Você acabou de implementar a versão fundacional do mesmo')
print('conceito que está por trás de:')
print()
print('  Apriori / FP-Growth     ← o que fizemos agora')
print('       ↓ escala e personalização')
print('  Collaborative Filtering  ← "quem comprou X também comprou Y"')
print('       ↓ fatores latentes')
print('  Matrix Factorization     ← técnica do Netflix Prize (2009)')
print('       ↓ deep learning')
print('  Neural Collaborative     ← estado da arte atual')
print()
print('A diferença: escala (milhões de usuários e itens)')
print('e personalização (cada usuário tem seu vetor latente).')
print()
print('O princípio é o mesmo: encontrar padrões de co-ocorrência.')